In [1]:

# True: not answer all question，False: answer all questions mandatorily。
answer_all = True
# load Bert
model_checkpoint = "distilbert-base-uncased"

batch_size = 16


In [2]:
# qa pairs
import pandas as pd
from datasets import Dataset, DatasetDict
with open('../testing_QA/reformatting_testing_qa.csv','r',encoding='utf-8') as f:
    df_c = pd.read_csv(f)


f:\Heidelberg\semester_5\INLPT\Final Project\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df_c

,question,answer,start_point,article_title,PMID,author,context,article_date
0,What is the purpose of the prospective study m...,To define the very early neuropsychological pr...,259,Neuropsychological profile in new-onset benign...,26667848,"Filippini M, Ardu E, Stefanelli S, Boni A, Gob...",PURPOSE: Increased evidence of subnormal neuro...,2015/12/16
1,How many children with BECTS were enrolled in ...,Fifteen school-aged children with BECTS.,438,Neuropsychological profile in new-onset benign...,26667848,"Filippini M, Ardu E, Stefanelli S, Boni A, Gob...",PURPOSE: Increased evidence of subnormal neuro...,2015/12/16
2,What neuropsychological domains showed signifi...,"Language, executive functions, and academic sk...",1041,Neuropsychological profile in new-onset benign...,26667848,"Filippini M, Ardu E, Stefanelli S, Boni A, Gob...",PURPOSE: Increased evidence of subnormal neuro...,2015/12/16
3,What implications are drawn from the results r...,Children with new-onset BECTS may demonstrate ...,2288,Neuropsychological profile in new-onset benign...,26667848,"Filippini M, Ardu E, Stefanelli S, Boni A, Gob...",PURPOSE: Increased evidence of subnormal neuro...,2015/12/16
4,What is the London Child Health and Developmen...,The London Child Health and Development Study ...,139,Toward earlier identification and preventative...,26670311,"Laurens KR, Cullen AE",PURPOSE: The London Child Health and Developme...,2015/12/17
5,What is the purpose of the London Child Health...,The purpose of the London Child Health and Dev...,325,Toward earlier identification and preventative...,26670311,"Laurens KR, Cullen AE",PURPOSE: The London Child Health and Developme...,2015/12/17
6,What does the London Child Health and Developm...,The London Child Health and Development Study ...,621,Toward earlier identification and preventative...,26670311,"Laurens KR, Cullen AE",PURPOSE: The London Child Health and Developme...,2015/12/17
7,What are the findings of the London Child Heal...,The findings of the London Child Health and De...,900,Toward earlier identification and preventative...,26670311,"Laurens KR, Cullen AE",PURPOSE: The London Child Health and Developme...,2015/12/17
8,What is the core hypothesis in developmental t...,genetic influences on intelligence and academi...,276,Large Cross-National Differences in Gene x Soc...,26671911,"Tucker-Drob EM, Bates TC",A core hypothesis in developmental theory pred...,2015/12/17
9,What did the meta-analysis by Tucker-Drob and ...,Gene x SES interaction on intelligence and aca...,287,Large Cross-National Differences in Gene x Soc...,26671911,"Tucker-Drob EM, Bates TC",A core hypothesis in developmental theory pred...,2015/12/17


In [4]:
pd.options.mode.copy_on_write = True

In [5]:
from copy import deepcopy
df_latest = deepcopy(df_c)

col_name=df_latest.columns.tolist() 
col_name.insert(0, 'id')

df_latest = df_latest.reindex(columns=col_name)  

for i in range(len(df_c['answer'])):
    new_context = f"{df_c['article_title'][i]}\n{df_c['author'][i]}\n{df_c['context'][i]}\n{df_c['article_date'][i]}\n{df_c['PMID'][i]}"
    if not df_c['start_point'][i].isnumeric():
        start_point = new_context.index(' '.join(df_c['start_point'][i].split()[0:2]))
        df_latest.replace({"start_point": {
                df_c['start_point'][i]: str([start_point])
            }}, inplace=True)
        temp_answer = {'text': [df_c['answer'][i]], 'answer_start': [start_point]}
    else:
        temp_answer = {'text': [df_c['answer'][i]], 'answer_start': [df_c['start_point'][i]]}
    df_latest.replace({"answer": {
                df_c['answer'][i]: str(temp_answer)
            }}, inplace=True)

    df_latest.replace({"context": {
                df_c['context'][i]: new_context
            }}, inplace=True)
    
    df_latest.at[i, 'id']= str(int(i))

C:\Users\Choi\AppData\Local\Temp\ipykernel_8796\3762812901.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_latest.at[i, 'id']= str(int(i))


In [6]:
df_latest

,id,question,answer,start_point,article_title,PMID,author,context,article_date
0,0,What is the purpose of the prospective study m...,{'text': ['To define the very early neuropsych...,259,Neuropsychological profile in new-onset benign...,26667848,"Filippini M, Ardu E, Stefanelli S, Boni A, Gob...",Neuropsychological profile in new-onset benign...,2015/12/16
1,1,How many children with BECTS were enrolled in ...,{'text': ['Fifteen school-aged children with B...,438,Neuropsychological profile in new-onset benign...,26667848,"Filippini M, Ardu E, Stefanelli S, Boni A, Gob...",Neuropsychological profile in new-onset benign...,2015/12/16
2,2,What neuropsychological domains showed signifi...,"{'text': ['Language, executive functions, and ...",1041,Neuropsychological profile in new-onset benign...,26667848,"Filippini M, Ardu E, Stefanelli S, Boni A, Gob...",Neuropsychological profile in new-onset benign...,2015/12/16
3,3,What implications are drawn from the results r...,{'text': ['Children with new-onset BECTS may d...,2288,Neuropsychological profile in new-onset benign...,26667848,"Filippini M, Ardu E, Stefanelli S, Boni A, Gob...",Neuropsychological profile in new-onset benign...,2015/12/16
4,4,What is the London Child Health and Developmen...,{'text': ['The London Child Health and Develop...,139,Toward earlier identification and preventative...,26670311,"Laurens KR, Cullen AE",Toward earlier identification and preventative...,2015/12/17
5,5,What is the purpose of the London Child Health...,{'text': ['The purpose of the London Child Hea...,325,Toward earlier identification and preventative...,26670311,"Laurens KR, Cullen AE",Toward earlier identification and preventative...,2015/12/17
6,6,What does the London Child Health and Developm...,{'text': ['The London Child Health and Develop...,621,Toward earlier identification and preventative...,26670311,"Laurens KR, Cullen AE",Toward earlier identification and preventative...,2015/12/17
7,7,What are the findings of the London Child Heal...,{'text': ['The findings of the London Child He...,900,Toward earlier identification and preventative...,26670311,"Laurens KR, Cullen AE",Toward earlier identification and preventative...,2015/12/17
8,8,What is the core hypothesis in developmental t...,{'text': ['genetic influences on intelligence ...,276,Large Cross-National Differences in Gene x Soc...,26671911,"Tucker-Drob EM, Bates TC",Large Cross-National Differences in Gene x Soc...,2015/12/17
9,9,What did the meta-analysis by Tucker-Drob and ...,{'text': ['Gene x SES interaction on intellige...,287,Large Cross-National Differences in Gene x Soc...,26671911,"Tucker-Drob EM, Bates TC",Large Cross-National Differences in Gene x Soc...,2015/12/17


In [7]:
# from datasets import DatasetDict
# dataset = DatasetDict.from_csv(
#         path_or_paths{'train': '../testing_QA/reformatting_testing_qa.csv'},
#         features: Optional[Features] = None,
#         cache_dir: str = None,
#         keep_in_memory: bool = False,
#         **kwargs,
#     )
# or we can do
# load_dataset("csv", data_files="my_file.csv")

In [8]:
training_df = df_latest.sample(frac=0.8, axis=0, replace=False, random_state=0)

In [9]:
testing_df = df_latest[~df_latest.index.isin(training_df.index)]

In [10]:
# construct datasets
training_data = Dataset.from_pandas(training_df, preserve_index=False)

In [11]:
training_data

Dataset({
    features: ['id', 'question', 'answer', 'start_point', 'article_title', 'PMID', 'author', 'context', 'article_date'],
    num_rows: 16
})

In [12]:
testing_data = Dataset.from_pandas(testing_df, preserve_index=False)

In [13]:
dataset = DatasetDict()
dataset['train'] = training_data
dataset['validation'] = testing_data

In [14]:
import numpy as np
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import default_data_collator
import torch
import collections
from tqdm.auto import tqdm
import ast

In [15]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [16]:
max_length = 384 # max length of input feature
doc_stride = 128 # repeat tokens number between 2 slices

In [17]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [18]:
tokenizer("What is your name?")

{'input_ids': [101, 2054, 2003, 2115, 2171, 1029, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [19]:
for i, example in enumerate(training_data):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = training_data[i]

In [20]:
len(tokenizer(example["question"], example["context"])["input_ids"])

416

In [21]:
tokenized_example = tokenizer(
    example["question"], # question
    example["context"],  # abstract
    max_length=max_length, 
    truncation="only_second", # slice only in the context
    return_overflowing_tokens=True,  # flag the slicing if exceed max length
    return_offsets_mapping=True,
    stride=doc_stride # slice step size
)

In [22]:
[len(x) for x in tokenized_example["input_ids"]]

[384, 181]

In [23]:
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 4), (5, 10), (11, 15), (16, 20), (21, 23), (24, 27), (28, 36), (37, 39), (40, 46), (47, 53), (54, 61), (61, 62), (63, 76), (77, 83), (84, 87), (88, 97), (98, 110), (110, 111), (0, 0), (0, 13), (14, 20), (21, 24), (25, 34), (35, 47), (48, 50), (51, 56), (57, 63), (64, 70), (71, 78), (78, 79), (80, 82), (82, 83), (83, 86), (86, 89), (90, 91), (91, 92), (93, 97), (97, 100), (101, 102), (103, 107), (108, 114), (115, 116), (116, 119), (120, 128), (129, 135), (135, 136), (137, 140), (141, 147), (147, 154), (154, 156), (157, 166), (167, 169), (170, 171), (172, 176), (177, 185), (185, 186), (187, 191), (192, 194), (194, 195), (196, 200), (201, 204), (205, 209), (210, 215), (216, 222), (223, 225), (226, 232), (233, 244), (245, 253), (253, 254), (255, 260), (261, 264), (265, 271), (272, 274), (275, 280), (281, 289), (290, 303), (304, 316), (317, 320), (321, 324), (325, 333), (334, 336), (337, 342), (343, 348), (349, 354), (355, 365), (365, 366), (367, 384), (385, 392), (393, 397), (

In [24]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [25]:
pad_on_right = tokenizer.padding_side == "right" #context on the right

In [26]:
def prepare_train_features(examples):
    # truncation and padding
    tokenized_examples = tokenizer(
        examples['question' if pad_on_right else "context"],
        examples['context' if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    offset_mapping = tokenized_examples.pop("offset_mapping")

    # tag data again
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # each slice processing
        # no answer sample tagged to CLS
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # distinguish question and context
        sequence_ids = tokenized_examples.sequence_ids(i)

        # original example index
        sample_index = sample_mapping[i]
        answers = examples["answer"][sample_index]
        # no answer, CLS location as answer
        if type(answers)==str:
            answers = ast.literal_eval(answers)
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # character level's start/end
            start_char = int(answers["answer_start"][0])
            end_char = start_char + len(answers["text"][0])

            # token level's index start
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # token level's index end
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # checking if exceeding length，if so, CLS index as tag
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Note: we could go after the last offset if the answer is the last word (edge case)
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [27]:
features = prepare_train_features(training_data[:5])


In [28]:
tokenized_datasets = dataset.map(prepare_train_features, batched=True, remove_columns=training_data.column_names)

Map: 100%|██████████| 4/4 [00:00<00:00, 469.88 examples/s]


In [29]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
args = TrainingArguments(
    f"test-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5, 
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3, 
    weight_decay=0.01,
)

In [31]:
from transformers import default_data_collator

data_collator = default_data_collator

In [32]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [33]:
trainer.train()

                                             
 33%|███▎      | 2/6 [00:13<00:24,  6.06s/it]

{'eval_loss': 5.997037887573242, 'eval_runtime': 0.9252, 'eval_samples_per_second': 6.485, 'eval_steps_per_second': 1.081, 'epoch': 1.0}


                                             
 67%|██████▋   | 4/6 [00:27<00:12,  6.48s/it]

{'eval_loss': 5.940711975097656, 'eval_runtime': 0.9211, 'eval_samples_per_second': 6.514, 'eval_steps_per_second': 1.086, 'epoch': 2.0}


                                             
100%|██████████| 6/6 [00:42<00:00,  7.16s/it]

{'eval_loss': 5.916805744171143, 'eval_runtime': 0.9249, 'eval_samples_per_second': 6.487, 'eval_steps_per_second': 1.081, 'epoch': 3.0}
{'train_runtime': 42.9656, 'train_samples_per_second': 1.815, 'train_steps_per_second': 0.14, 'train_loss': 5.874773661295573, 'epoch': 3.0}


TrainOutput(global_step=6, training_loss=5.874773661295573, metrics={'train_runtime': 42.9656, 'train_samples_per_second': 1.815, 'train_steps_per_second': 0.14, 'train_loss': 5.874773661295573, 'epoch': 3.0})

In [34]:
trainer.save_model("test-qa-trained")

## Evaluation

In [35]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()


odict_keys(['loss', 'start_logits', 'end_logits'])

In [36]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([6, 384]), torch.Size([6, 384]))

## Testing Answer

In [37]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([183,  80, 186,  81, 349, 347]),
 tensor([334, 306, 337, 152, 106, 224]))

In [38]:
n_best_size = 20

In [39]:
def prepare_validation_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [40]:
f_t = prepare_validation_features(testing_data[:5])

In [41]:
validation_features = dataset["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=dataset["validation"].column_names
)

Map: 100%|██████████| 4/4 [00:00<00:00, 347.38 examples/s]


In [42]:
raw_predictions = trainer.predict(validation_features)

100%|██████████| 1/1 [00:00<00:00, 1001.98it/s]


In [43]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [44]:
max_answer_length = 30

In [51]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = dataset["validation"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]


KeyError: "Invalid key: 0. Please first select a split. For example: `my_dataset_dictionary['train'][0]`. Available splits: ['train', 'validation']"

In [46]:
valid_answers

[{'score': 1.2033025, 'text': 'tient equal or superior to 85, in order to'},
 {'score': 1.159085, 'text': 'tient equal or superior to'},
 {'score': 1.1112173,
  'text': 'tient equal or superior to 85, in order to exclude additional effects on the neuropsycho'},
 {'score': 1.0925677, 'text': 'tient equal or superior to 85, in'},
 {'score': 0.90877676,
  'text': '<0.05 was considered significant. RESULTS: Significant differences between patients and controls emerged with respect to'},
 {'score': 0.8879608,
  'text': 'considered significant. RESULTS: Significant differences between patients and controls emerged with respect to'},
 {'score': 0.8861419,
  'text': '<85% and with a Performance Intelligence Quotient equal or superior to 85, in order to'},
 {'score': 0.85607976, 'text': 'assessment'},
 {'score': 0.8542516,
  'text': 'define the very early neuropsychological profile of children with benign epilepsy'},
 {'score': 0.8475945, 'text': 'profile of children with benign epilepsy'},
 {'

In [47]:
dataset["validation"][0]['question']

'What is the purpose of the prospective study mentioned in the article?'

In [48]:
dataset["validation"][0]["answer"]

"{'text': ['To define the very early neuropsychological profile of children with benign epilepsy with centrotemporal spikes (BECTS), including executive functions (EF).'], 'answer_start': ['259']}"